# Multiple Regression Analysis: Further Issues

In [1]:
import pandas as pd
import numpy as np
import wooldridge
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [51]:
def duan_smearing_estimator(model):
    """
    Compute Duan's smearing estimate (sigma0) from a fitted log-linear model.

    Parameters:
    model: A fitted statsmodels OLS model where the dependent variable is log(y).

    Returns:
    float: Smearing estimate sigma0.
    """
    resid = model.resid  # residuals from log-linear model
    sigma0 = np.mean(np.exp(resid))  # Duan's smearing estimate
    return sigma0

In [2]:
wooldridge.data()

  J.M. Wooldridge (2019) Introductory Econometrics: A Modern Approach,
  Cengage Learning, 6th edition.

  401k       401ksubs    admnrev       affairs     airfare
  alcohol    apple       approval      athlet1     athlet2
  attend     audit       barium        beauty      benefits
  beveridge  big9salary  bwght         bwght2      campus
  card       catholic    cement        census2000  ceosal1
  ceosal2    charity     consump       corn        countymurders
  cps78_85   cps91       crime1        crime2      crime3
  crime4     discrim     driving       earns       econmath
  elem94_95  engin       expendshares  ezanders    ezunem
  fair       fertil1     fertil2       fertil3     fish
  fringe     gpa1        gpa2          gpa3        happiness
  hprice1    hprice2     hprice3       hseinv      htv
  infmrt     injury      intdef        intqrt      inven
  jtrain     jtrain2     jtrain3       kielmc      lawsch85
  loanapp    lowbrth     mathpnl       meap00_01   meap01
  meap93    

## Examples

In [3]:
pd.options.display.float_format = '{:.4f}'.format

In [39]:
hprice2 = wooldridge.data("hprice2")
attend = wooldridge.data("attend")
ceosal1 = wooldridge.data('ceosal1')
gpa2 = wooldridge.data('gpa2')
ceosal2 = wooldridge.data('ceosal2')

### 6.1 Effects of Pollution on Housing Prices

To make beta coefficients, I use `StandardScaler` from sklearn library

In [6]:
wooldridge.data("hprice2", description=True)

name of dataset: hprice2
no of variables: 12
no of observations: 506

+----------+-------------------------------+
| variable | label                         |
+----------+-------------------------------+
| price    | median housing price, $       |
| crime    | crimes committed per capita   |
| nox      | nit ox concen; parts per 100m |
| rooms    | avg number of rooms           |
| dist     | wght dist to 5 employ centers |
| radial   | access. index to rad. hghwys  |
| proptax  | property tax per $1000        |
| stratio  | average student-teacher ratio |
| lowstat  | perc of people 'lower status' |
| lprice   | log(price)                    |
| lnox     | log(nox)                      |
| lproptax | log(proptax)                  |
+----------+-------------------------------+

D. Harrison and D.L. Rubinfeld (1978), “Hedonic Housing Prices and the
Demand for Clean Air,” by Harrison, D. and D.L.Rubinfeld, Journal of
Environmental Economics and Management 5, 81-102. Diego Garcia, a
for

In [40]:
hprice2.head()

,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,24000.000,0.006,5.380,6.570,4.090,1,29.600,15.300,4.980,10.086,1.683,5.690
1,21599.000,0.027,4.690,6.420,4.970,2,24.200,17.800,9.140,9.980,1.545,5.489
2,34700.000,0.027,4.690,7.180,4.970,2,24.200,17.800,4.030,10.454,1.545,5.489
3,33400.000,0.032,4.580,7.000,6.060,3,22.200,18.700,2.940,10.416,1.522,5.403
4,36199.000,0.069,4.580,7.150,6.060,3,22.200,18.700,5.330,10.497,1.522,5.403


In [7]:
df = hprice2.copy()
vars_to_standardize = ["price", "nox", "crime", "rooms", "dist", "stratio"]
df[vars_to_standardize] = StandardScaler().fit_transform(df[vars_to_standardize])

In [8]:
df.head()

,price,crime,nox,rooms,dist,radial,proptax,stratio,lowstat,lprice,lnox,lproptax
0,0.1618,-0.4201,-0.1467,0.4074,0.1398,1,29.6000,-1.4601,4.9800,10.0858,1.6827,5.6904
1,-0.0992,-0.4177,-0.7430,0.1937,0.5581,2,24.2000,-0.3047,9.1400,9.9804,1.5454,5.4889
2,1.3249,-0.4177,-0.7430,1.2765,0.5581,2,24.2000,-0.3047,4.0300,10.4545,1.5454,5.4889
3,1.1836,-0.4171,-0.8380,1.0200,1.0761,3,22.2000,0.1113,2.9400,10.4163,1.5217,5.4027
4,1.4878,-0.4128,-0.8380,1.2337,1.0761,3,22.2000,0.1113,5.3300,10.4968,1.5217,5.4027


In [9]:
pd.set_option("display.float_format", "{:.3f}".format)
# pd.reset_option('display.float_format')

In [10]:
model01 = smf.ols("price ~ nox + crime + rooms + dist + stratio", data=df).fit()
model01.summary2().tables[1].iloc[:, :2]

,Coef.,Std.Err.
Intercept,0.000,0.027
nox,-0.340,0.045
crime,-0.143,0.031
rooms,0.514,0.030
dist,-0.235,0.043
stratio,-0.270,0.030


One standard deviation in $nox$ decreases price by 0.34 standard deviations.  
One standard deviation in $crime$ decreases price by 0.14 standard deviations.  
nox affects price more than crime  
t statistic does not change if we standardize data

### 6.2 Effects of Pollution on Housing Prices

In [11]:
model02 = smf.ols(
    "lprice ~ lnox + np.log(dist) + rooms + I(rooms**2) + stratio", data=hprice2
).fit()
model02.summary2().tables[1].iloc[:, :4]

,Coef.,Std.Err.,t,P>|t|
Intercept,13.385,0.566,23.630,0.000
lnox,-0.902,0.115,-7.862,0.000
np.log(dist),-0.087,0.043,-2.005,0.045
rooms,-0.545,0.165,-3.295,0.001
I(rooms ** 2),0.062,0.013,4.862,0.000
stratio,-0.048,0.006,-8.129,0.000


$rooms^2$ is statistically significant. Coefficient on $rooms$ is negative and $rooms^2$ is positive.  
Rooms have negative effect on price until some point, the effect becomes positive. To get this point
$$ 0.545/[2(0.062)] \approx 4.4 $$

3 rooms and less decreasing price is not practical, the negative result is due to the sample containing few observations with 3 rooms and less

Effect of adding one extra room
$$ \% \Delta price \approx 100[-0.545 + 2(0.062)]rooms]\Delta rooms$$
which is equal to
$$ \% \Delta price  = [-54.5 + 12.4 \, rooms] \Delta rooms$$

Note: effect of rooms depends on number of rooms, so we can't look at the coefficients alone

### 6.3 Effects of Attendance on Final Exam Performance

In [12]:
wooldridge.data("attend", description=True)

name of dataset: attend
no of variables: 11
no of observations: 680

+----------+------------------------------+
| variable | label                        |
+----------+------------------------------+
| attend   | classes attended out of 32   |
| termGPA  | GPA for term                 |
| priGPA   | cumulative GPA prior to term |
| ACT      | ACT score                    |
| final    | final exam score             |
| atndrte  | percent classes attended     |
| hwrte    | percent homework turned in   |
| frosh    | =1 if freshman               |
| soph     | =1 if sophomore              |
| missed   | number of classes missed     |
| stndfnl  | (final - mean)/sd            |
+----------+------------------------------+

These data were collected by Professors Ronald Fisher and Carl
Liedholm during a term in which they both taught principles of
microeconomics at Michigan State University. Professors Fisher and
Liedholm kindly gave me permission to use a random subset of their
data, and 

In [13]:
model03 = smf.ols(
    "stndfnl ~ atndrte + priGPA + ACT + " \
    "I(priGPA**2)+ I(ACT**2)+ I(priGPA*atndrte)",
    data=attend,
).fit()
model03.summary2().tables[1].iloc[:, :4]

,Coef.,Std.Err.,t,P>|t|
Intercept,2.050,1.360,1.507,0.132
atndrte,-0.007,0.010,-0.656,0.512
priGPA,-1.629,0.481,-3.386,0.001
ACT,-0.128,0.098,-1.300,0.194
I(priGPA ** 2),0.296,0.101,2.928,0.004
I(ACT ** 2),0.005,0.002,2.083,0.038
I(priGPA * atndrte),0.006,0.004,1.294,0.196


When $priGPA = 0$, attendance has a negative effect on exam score  
 t statistic is misleading. Check the F test

In [14]:
model03.params.index

Index(['Intercept', 'atndrte', 'priGPA', 'ACT', 'I(priGPA ** 2)',
       'I(ACT ** 2)', 'I(priGPA * atndrte)'],
      dtype='object')

In [15]:
model03.f_test("atndrte = 0, I(priGPA * atndrte) = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=4.319021763197198, p=0.013683848684002065, df_denom=673, df_num=2>

To estimate partial effect of $atndrte$, plug values for $priorGPA$ in the interaction term

In [16]:
attend['priGPA'].mean()

np.float64(2.586774999078582)

Effect of $atndrte$ is
$$ -0,0067 + 0.0056(2.59) \approx 0.0078$$  
if attendance rate increases by 10 percentage points, final exam score increase by 0.078 standard deviations  
To test if the 0.0078 is significant from zero

In [17]:
model032 = smf.ols(
    "stndfnl ~ atndrte + priGPA + ACT + " \
    "I(priGPA**2)+ I(ACT**2)+ I((priGPA-2.59)*atndrte)",
    data=attend,
).fit()
model032.summary2().tables[1].iloc[:, :4]

,Coef.,Std.Err.,t,P>|t|
Intercept,2.050,1.360,1.507,0.132
atndrte,0.008,0.003,2.938,0.003
priGPA,-1.629,0.481,-3.386,0.001
ACT,-0.128,0.098,-1.300,0.194
I(priGPA ** 2),0.296,0.101,2.928,0.004
I(ACT ** 2),0.005,0.002,2.083,0.038
I((priGPA - 2.59) * atndrte),0.006,0.004,1.294,0.196


Using this method, $atndrte$ coefficient changes to our calculated 0.0078, with p value of 0.0034, statistically significant

### 6.4 CEO Compensation and Firm Performance

In [18]:
wooldridge.data('ceosal1', description = True)

name of dataset: ceosal1
no of variables: 12
no of observations: 209

+----------+-------------------------------+
| variable | label                         |
+----------+-------------------------------+
| salary   | 1990 salary, thousands $      |
| pcsalary | % change salary, 89-90        |
| sales    | 1990 firm sales, millions $   |
| roe      | return on equity, 88-90 avg   |
| pcroe    | % change roe, 88-90           |
| ros      | return on firm's stock, 88-90 |
| indus    | =1 if industrial firm         |
| finance  | =1 if financial firm          |
| consprod | =1 if consumer product firm   |
| utility  | =1 if transport. or utilties  |
| lsalary  | natural log of salary         |
| lsales   | natural log of sales          |
+----------+-------------------------------+

I took a random sample of data reported in the May 6, 1991 issue of
Businessweek.


In [19]:
model04 = smf.ols('salary ~ sales + roe', data = ceosal1).fit()
model04.rsquared, model04.rsquared_adj

(np.float64(0.029171686607148084), np.float64(0.019746169001392255))

In [20]:
model042 = smf.ols('lsalary ~ lsales + roe', data = ceosal1).fit()
model042.rsquared, model042.rsquared_adj

(np.float64(0.28198874478633673), np.float64(0.2750177617260099))

The second model has higher $R^2$ and $\bar R^2$ because variation in $lsalary$ is lower due to taking the $\log$

In [21]:
y = model04.model.endog
y_bar = np.mean(y) 
TSS = np.sum((y - y_bar) ** 2)

In [22]:
y = model042.model.endog
y_bar = np.mean(y) 
TSS2 = np.sum((y - y_bar) ** 2)

In [23]:
TSS, TSS2

(np.float64(391732982.00956935), np.float64(66.72216321299874))

### 6.5 Confidence Interval for Predicted College GPA

In [24]:
wooldridge.data('gpa2', description= True)

name of dataset: gpa2
no of variables: 12
no of observations: 4137

+----------+----------------------------------+
| variable | label                            |
+----------+----------------------------------+
| sat      | combined SAT score               |
| tothrs   | total hours through fall semest  |
| colgpa   | GPA after fall semester          |
| athlete  | =1 if athlete                    |
| verbmath | verbal/math SAT score            |
| hsize    | size grad. class, 100s           |
| hsrank   | rank in grad. class              |
| hsperc   | high school percentile, from top |
| female   | =1 if female                     |
| white    | =1 if white                      |
| black    | =1 if black                      |
| hsizesq  | hsize^2                          |
+----------+----------------------------------+

For confidentiality reasons, I cannot provide the source of these
data. I can say that  they come from a midsize research university
that also supports men’s and w

In [25]:
model05 = smf.ols('colgpa ~ sat + hsperc + hsize + hsizesq', data = gpa2).fit()
model05.summary2().tables[1].iloc[:, :2]

,Coef.,Std.Err.
Intercept,1.493,0.075
sat,0.001,0.000
hsperc,-0.014,0.001
hsize,-0.061,0.017
hsizesq,0.005,0.002


In [26]:
model05.summary2().tables[0]

,0,1,2,3
0,Model:,OLS,Adj. R-squared:,0.277
1,Dependent Variable:,colgpa,AIC:,6945.8642
2,Date:,2025-07-29 13:43,BIC:,6977.5029
3,No. Observations:,4137,Log-Likelihood:,-3467.9
4,Df Model:,4,F-statistic:,398.0
5,Df Residuals:,4132,Prob (F-statistic):,2.13e-290
6,R-squared:,0.278,Scale:,0.31345


In [27]:
se = model05.mse_resid ** 0.5
se

np.float64(0.5598638444189318)

We can use the model to predict GPA at the values of the independent variables $SAT = 1200, hsperc = 30, hsize = 5$

In [28]:
new_data = pd.DataFrame({
    'sat': [1200],
    'hsperc': [30],
    'hsize': [5],
    'hsizesq': [25]
})

In [29]:
predicted_gpa = model05.predict(new_data)
predicted_gpa

0   2.700
dtype: float64

To calculate confidence interval for our predicted $2.7$, we subtract the values used for the prediction from the independent variables

In [30]:
df = gpa2.copy()
df['sat'] = gpa2['sat']-1200
df['hsize'] = gpa2['hsize']-5
df['hsperc'] = gpa2['hsperc']-30
df['hsizesq'] = gpa2['hsizesq']-25

In [31]:
model052 = smf.ols('colgpa ~ sat + hsperc + hsize + hsizesq', data = df).fit()
model052.summary2().tables[1].iloc[:]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,2.700,0.020,135.833,0.000,2.661,2.739
sat,0.001,0.000,22.886,0.000,0.001,0.002
hsperc,-0.014,0.001,-24.698,0.000,-0.015,-0.013
hsize,-0.061,0.017,-3.690,0.000,-0.093,-0.029
hsizesq,0.005,0.002,2.406,0.016,0.001,0.010


We get same coefficients as before except for the intercept value.  The confidence interval for the prediction is [2.66,2.74]

In [32]:
from statsmodels.iolib.summary2 import summary_col

summary = summary_col(
    [model05, model052],
    stars=True,
    model_names=["Uncentered", "Centered"],
    info_dict={
        "R-squared": lambda x: f"{x.rsquared:.4f}",
        "No. observations": lambda x: f"{int(x.nobs)}",
    },
)

print(summary)


                 Uncentered  Centered 
--------------------------------------
Intercept        1.4927***  2.7001*** 
                 (0.0753)   (0.0199)  
sat              0.0015***  0.0015*** 
                 (0.0001)   (0.0001)  
hsperc           -0.0139*** -0.0139***
                 (0.0006)   (0.0006)  
hsize            -0.0609*** -0.0609***
                 (0.0165)   (0.0165)  
hsizesq          0.0055**   0.0055**  
                 (0.0023)   (0.0023)  
R-squared        0.2781     0.2781    
R-squared Adj.   0.2774     0.2774    
No. observations 4137       4137      
R-squared        0.2781     0.2781    
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


Note: centering $hsize$ and then squaring it results in a different value than squaring $hsize$ and then centering it  
The correct order is: $$square \rightarrow center$$

### 6.6 Confidence Interval for Future College GPA

To get prediction confidence intervals

In [33]:
model06 = smf.ols('colgpa ~ sat + hsperc + hsize + hsizesq', data = gpa2).fit()
model06.summary2().tables[1].iloc[:, :2]

,Coef.,Std.Err.
Intercept,1.493,0.075
sat,0.001,0.000
hsperc,-0.014,0.001
hsize,-0.061,0.017
hsizesq,0.005,0.002


In [34]:
new_data = pd.DataFrame({
    'sat': [1200],
    'hsperc': [30],
    'hsize': [5],
    'hsizesq': [25]
})

In [35]:
pred = model06.get_prediction(new_data)
pred.summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,2.700,0.020,2.661,2.739,1.602,3.798


The confidence interval for the prediction is [1.6, 3.8]. Its wider because it accounts for individual specific variation [residual variation] due to unobserved factors

Note: in Example 6.5, we got confidence interval for average college GPA for students with the given characteristics.  
In 6.6, we added residual variation

### 6.7 Predicting CEO Salaries

In [38]:
wooldridge.data('ceosal2', description= True)

name of dataset: ceosal2
no of variables: 15
no of observations: 177

+----------+--------------------------------+
| variable | label                          |
+----------+--------------------------------+
| salary   | 1990 compensation, $1000s      |
| age      | in years                       |
| college  | =1 if attended college         |
| grad     | =1 if attended graduate school |
| comten   | years with company             |
| ceoten   | years as ceo with company      |
| sales    | 1990 firm sales, millions      |
| profits  | 1990 profits, millions         |
| mktval   | market value, end 1990, mills. |
| lsalary  | log(salary)                    |
| lsales   | log(sales)                     |
| lmktval  | log(mktval)                    |
| comtensq | comten^2                       |
| ceotensq | ceoten^2                       |
| profmarg | profits as % of sales          |
+----------+--------------------------------+

See CEOSAL1.RAW


In [41]:
model07 = smf.ols('lsalary ~ lsales + lmktval + ceoten', data = ceosal2).fit()
model07.summary2().tables[1].iloc[:,:2]

,Coef.,Std.Err.
Intercept,4.504,0.257
lsales,0.163,0.039
lmktval,0.109,0.050
ceoten,0.012,0.005


In [42]:
resid = model07.resid
n = model07.nobs

The Duan Smeaing estimate is $$n^{-1} \sum \exp(\hat u)$$

In [48]:
sigma0 = np.sum(np.exp(resid))/n
sigma0

np.float64(1.1356613266630768)

In [52]:
duan_smearing_estimator(model07)

np.float64(1.1356613266630768)

The prediction for $sales = 5000, mktval = 10000, ceoten = 10$

In [46]:
new_data = pd.DataFrame({
    'lsales': [np.log(5000)],
    'lmktval': [np.log(10000)],
    'ceoten': [10]
})

In [49]:
prediction = model07.predict(new_data)
prediction

0   7.014
dtype: float64

The predicted value is $$\exp(y) * \hat \sigma_0$$

In [50]:
np.exp(prediction)* sigma0

0   1263.059
dtype: float64

### 6.8 Predicting CEO Salaries

In [53]:
model08 = smf.ols('lsalary ~ lsales + lmktval + ceoten', data = ceosal2).fit()
model08.summary2().tables[1].iloc[:,:2]

,Coef.,Std.Err.
Intercept,4.504,0.257
lsales,0.163,0.039
lmktval,0.109,0.050
ceoten,0.012,0.005


To get $salary$ from our log model, we take the exponential $$\hat m = \exp{\widehat{lsalary}}$$ 

We call it $\hat m$ to avoid the confusion with $\hat y$ that is predicted from the level model

In [58]:
mhat = np.exp(model08.predict())
df = ceosal2.copy()
df['mhat'] = mhat

Then get the correlation between it and y and square it

In [61]:
corr = df['mhat'].corr(df['salary'])
np.square(corr)

np.float64(0.2430807795867979)

The log model explains 24.3% of the variation in $salary$

In [62]:
model082= smf.ols('salary ~ sales + mktval + ceoten', data = ceosal2).fit()
model082.rsquared

np.float64(0.20127439139676495)

The level model explains 20% of the variation in $salary$ while the log model explains $24.3%$  
The log model is preferred to the level model

## 🚧Computer Exercises